# TruLens-Canopy Quickstart

 Canopy is an open-source framework and context engine built on top of the Pinecone vector database so you can build and host your own production-ready chat assistant at any scale. By integrating TruLens into your Canopy assistant, you can quickly iterate on and gain confidence in the quality of your chat assistant.

## Set Keys

In [ ]:
import os

os.environ["PINECONE_API_KEY"] = "..."
os.environ["OPENAI_API_KEY"] = "sk-..."

## Load data

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_parquet("https://storage.googleapis.com/pinecone-datasets-dev/pinecone_docs_ada-002/raw/file1.parquet")
data.head()

## Setup Tokenizer

In [ ]:
from canopy.tokenizer import Tokenizer
Tokenizer.initialize()

tokenizer = Tokenizer()

tokenizer.tokenize("Hello world!")

## Create and Load Index

In [ ]:
from canopy.knowledge_base import KnowledgeBase

INDEX_NAME = "my-index"

kb = KnowledgeBase(index_name=INDEX_NAME)

from canopy.knowledge_base import list_canopy_indexes
if not any(name.endswith(INDEX_NAME) for name in list_canopy_indexes()):
    kb.create_canopy_index()

kb = KnowledgeBase(index_name=INDEX_NAME)
kb.connect()

from canopy.models.data_models import Document

documents = [Document(**row) for _, row in data.iterrows()]

from tqdm.auto import tqdm

batch_size = 10

for i in tqdm(range(0, len(documents), batch_size)):
    kb.upsert(documents[i: i+batch_size])

## Create context and chat engine

In [ ]:
import json
from canopy.models.data_models import Query
from canopy.context_engine import ContextEngine
context_engine = ContextEngine(kb)

from canopy.chat_engine import ChatEngine
chat_engine = ChatEngine(context_engine)

## Instrument static methods used by engine with TruLens 

In [ ]:
from trulens_eval.tru_custom_app import instrument

from canopy.context_engine import ContextEngine
instrument.method(ContextEngine, "query")

from canopy.chat_engine import ChatEngine
instrument.method(ChatEngine, "chat")

from canopy.chat_engine.query_generator.base import QueryGenerator
instrument.method(QueryGenerator, "generate")

## Create feedback functions using instrumented methods

In [ ]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np

# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

intput = Select.RecordCalls.chat.args.messages[0].content
context = Select.RecordCalls.context_engine.query.rets.content.root[:].snippets[:].text
output = Select.RecordCalls.chat.rets.choices[0].message.content

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness", higher_is_better=True)
    .on(context.collect())
    .on(output)
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance", higher_is_better=True)
    .on(intput)
    .on(output)
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance", higher_is_better=True)
    .on(intput)
    .on(context)
    .aggregate(np.mean)
)

In [ ]:
from trulens_eval import TruCustomApp
tru_recorder = TruCustomApp(chat_engine, feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

In [ ]:
from canopy.models.data_models import Messages, UserMessage

queries = [
    [UserMessage(content="How can you get started with Pinecone and TruLens?")],
    [UserMessage(content="What are different ways to improve retrieval?")],
    [UserMessage(content="How latency requirements does Pinecone support?")]
]

for query in queries:
    with tru_recorder as recording:
        chat_engine.chat(query)

In [ ]:
from trulens_eval import Tru
tru = Tru()
tru.run_dashboard()

In [ ]:
tru.get_leaderboard(app_ids=[])